In [7]:
%load_ext autoreload
%autoreload 2
from data_loader import load_datasets, create_data_loaders
import torch
from training_pipeline import train_with_different_parameters, plot_results, train_with_augmentation
from training_pipeline import repeat_training
import os
from cnn_model import Net
import numpy as np
import random

SEED = 42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [9]:
set_seed(SEED)
# optimal parameters
batch_size = 128
learning_rate = 0.0005
dropout = 0.4
betas = (0.7, 0.997)
weight_decay = 1e-5

In [ ]:
augs = ['rotation', 'blur', 'bright', 'cutout']
for i in range(4):
    train, val, test = load_datasets((32, 32), apply_rotation=i==0,
                                                        apply_blur = i==1,
                                                        apply_brightness= i==2,
                                                        apply_cutout = i==3)
    train_dat, val_dat, test_dat = create_data_loaders(train, val, test, batch_size)
    aug = augs[i]
    set_seed(SEED)
    train_with_augmentation(5, Net, 10, train_dat, val_dat, test_dat, 'cuda', batch_size, lr=learning_rate, drop=dropout, beta=betas, aug=aug, weight_decay=weight_decay)